# Manipulate data the MXNet way with `ndarray`

It's impossible to get anything done if we can't manipulate data. So let's start by introducing NDArrays, MXNet's primary tool for storing and transforming data. If you've worked with NumPy before, you'll notice that NDArrays are by design similar to NumPy's multi-dimensional array. However, they confer a few key advantages. First, NDArrays support asynchronous computation on CPU, GPU, and distributed cloud architectures. Second, they provide support for automatic differentiation. These properties make NDArray an ideal library for machine learning, both for researchers and engineers launching production systems.


## Getting started

In this chapter, we'll get you going with the basic functionality. Don't worry if you don't understand any of the basic math, like element-wise operations or normal distributions. In the next two chapters we'll take another pass at NDArray, teaching you both the math you'll need and how to realize it in code.

To get started, let's import `ndarray` from `mxnet`.

In [ ]:
from mxnet import nd

Next, let's see how to create an NDArray, without values initialized to 1. Specifically 
we'll create a 2D array (also called a *matrix*) with 2 rows and 3 columns.

In [ ]:
x = nd.ones(shape=(2,3))
x

Often, we'll want to create arrays whose values are sampled randomly. This is especially common when we intend to use the array as a parameter in a neural network. In this snippet, we initialize with values drawn from a standard normal distribution.

In [ ]:
y = nd.random_normal(shape=(2,3))
y

As in NumPy, the dimensions of each NDArray are accessible via the `.shape` attribute.

In [ ]:
y.shape

We can also query its size, which is equal to the product of the components of the shape. Together with the precision of the stored values, this tells us how much memory the array occupies.

In [ ]:
y.size

## Operations

NDarray supports a large number of standard mathematical operations. Such as elementwise operations

In [ ]:
x + y

In [ ]:
x * y

In [ ]:
nd.exp(y)

Transpose and matrix-matrix multiplication

In [ ]:
nd.dot(x, y.T)

We will present more operators on the [linear algebra](P01-C03-linear-algebra.ipynb) tutorial.

## In-place operations

In the previous example, we allocated new memory to host the results of operations. Even we write `y = x + y`, we will replace `y` with the new created memory. In Python, we can check the object a variable refers to through `id`.

In [ ]:
print('id(y):', id(y))
y = y + x
print('id(y):', id(y))
print(y)

To make better use of memory, we can perform operations in place, reusing already allocated memory. We can specify where to write the results of operations by assigning them with slice notation, e.g., `result[:] = ...`.

In [ ]:
z = nd.zeros_like(x)
print('id(z):', id(z))
z[:] = x + y
print('id(z):', id(z))
print(z)

If we're not planning to re-use ``x``, then we can assign the result to ``x`` itself.

If the input to the operator will not be used later, we can reuse its memory through assignment operators. 

In [ ]:
print('id(y):', id(y))
y += x
print('id(y):', id(y))
print(y)

## Slicing

MXNet NDArrays support slicing in all the ridiculous ways you might imagine accessing your data. Here's an example of reading the second and third rows from ``x``.

In [ ]:
x[1:2]

Now let's try writing to a specific element.

In [ ]:
x[1,2] = 9.0
x

Multi-dimensional slicing is also supported.

In [ ]:
x[1:2,1:3]

In [ ]:
x[1:2,1:3] = 5.0
x

## Broadcasting

You might wonder, what happens if you add a vector ``y`` to a matrix ``X``? These operations, where we compose a low dimensional array ``y`` with a high-dimensional array ``X`` invoke a functionality called broadcasting. Here, the low-dimensional array is duplicated along any axis with dimension ``1`` to match the shape of the high dimesnional array. Consider the following example.

In [ ]:
X = nd.ones(shape=(3,3))
print('X = ', X)
y = nd.arange(3)
print('y = ', y)
print('X + y = ', X+y)

While `y` is initially of shape (3), MXNet infers its shape to be (1,3), and then broadcasts along the rows to form a (3,3) matrix). You might wonder, why did MXNet choose to interpret `y` as a (1,3) matrix and not (3,1). That's because broadcasting prefers to duplicate along the left most axis. We can alter this behavior by explicitly giving `y` a 2D shape.

In [ ]:
y = y.reshape((3,1))
print('y = ', y)
print('X + y = ', X+y)

## Converting from MXNet NDArray to NumPy 

Converting MXNet NDArrays to and from NumPy is easy. The converted arrays do not share memory.

In [ ]:
a = x.asnumpy()
type(a)

In [ ]:
y = nd.array(a) 
y

## Managing context

By far, MXNet NDArray looks almost identical to NumPy. One of the key features make MXNet differs to NumPy is the supporting for various hardware.

In MXNet, every array has a context. One context could be the CPU. Other contexts might be various GPUs. Things can get even hairier when we deploy jobs across multiple servers. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model's parameters to live on the GPU. To start, let's try initializing an array on the first GPU.

In [ ]:
from mxnet import gpu
z = nd.ones(shape=(2,3), ctx=gpu(0))
z

Given an NDArray on a given context, we can copy it to another context by using the ``copyto()`` method.

In [ ]:
x_gpu = x.copyto(gpu(0))
print(x_gpu)

The result of an operator will has the same context as the inputs.

In [ ]:
x_gpu + z

## Watch out!

Imagine that your variable ``z`` already lives on your second GPU (``gpu(0)``). What happens if we call ``z.copyto(gpu(0))``? It will make a copy and allocate new memory, even though that variable already lives on the desired device! 

Often, we only want to make a copy if the variable *currently* lives in the wrong context. In these cases, we can call ``as_in_context()``. If the variable is already on ``gpu(0)`` then this is a no-op.

In [ ]:
print('id(z):', id(z))
z = z.copyto(gpu(0))
print('id(z):', id(z))
z = z.as_in_context(gpu(0))
print('id(z):', id(z))
print(z)

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)